<h2>Scraping multiple products on Amazon</h2>

In [1]:
# Imports
from bs4 import BeautifulSoup
import requests
import pandas as pd

# Loading environment variable
import os
from dotenv import load_dotenv
load_dotenv()


True

<h3>Functions to extract meta data</h3>

In [2]:
def get_product_name(soup):
  try:
    title = soup.find('span', attrs={'id':'productTitle'})
    title_value = title.text
    title_string = title_value.strip()

  except AttributeError:
    title_string = ''
  
  return title_string
#------------------------------------------------------#
def get_product_price(soup):
  try:
    price = soup.find('span', attrs={'id':'priceblock_ourprice'}).string.strip()

  except AttributeError:
    try:
      price = soup.find('span', attrs={'id':'priceblock_dealprice'}).string.strip()
    except:
      price = ''

  return price
#------------------------------------------------------#
def get_product_rating(soup):
  try:
    rating = soup.find('span', attrs={'data-hook':'rating-out-of-text'}).text
  except AttributeError:
    try:
      rating = soup.find('span', attrs={'class':'a-icon-alt'}).text
    except:
      rating = ''
  return rating
#------------------------------------------------------#
def get_available_stock(soup):
  try:
    available_stock = soup.find('div', attrs={'class': 'a-section a-spacing-base a-spacing-top-micro'}).string.strip()
  except AttributeError:
    available_stock = 'No Stock'
  
  return available_stock

In [3]:
if __name__ == '__main__':
  #User_Agent = os.environ.get('USER_AGENT')
  HEADERS = ({'User-Agent': '', 'Accept-Language': 'en-US, en;q=0.5'})    
      # removed User_Agent -- removing makes it work , more research needed

  # URL is based on search results page
  URL = 'https://www.amazon.com/s?k=fidget+spinner&ref=nb_sb_noss_2'

  webpage = requests.get(URL, headers=HEADERS)    # requesting HTTP of full page

  search_results = BeautifulSoup(webpage.content, 'html.parser')

  links = search_results.find_all('a', attrs={'class':'a-link-normal s-no-outline'})
  link_list = []
  for link in links:
    link_list.append(link.get('href'))

  product_detail = {'title':[], 'price':[], 'rating':[], 'availability':[]}

  for link in link_list:
    updated_webpage = requests.get('https://www.amazon.com' + link, headers=HEADERS)

    new_search_results = BeautifulSoup(updated_webpage.content, 'html.parser')

    product_detail['title'].append(get_product_name(new_search_results))
    product_detail['price'].append(get_product_price(new_search_results))
    product_detail['rating'].append(get_product_rating(new_search_results))
    product_detail['availability'].append(get_available_stock(new_search_results))

  amazon_product_df = pd.DataFrame.from_dict(product_detail)
  amazon_product_df.to_csv('amazon_fidget_spinner.csv', header=True, index=False)

In [4]:
#search_results
#new_search_results
amazon_product_df

,title,price,rating,availability
0,"FLYNOVA Flying Toys for Age 8-13,2023 Magic Ho...",,4 out of 5,No Stock
1,"SCIONE Fidget Spinners Toys 5 Pack, Sensory Ha...",,4.5 out of 5,No Stock
2,ATESSON Fidget Spinner Toy Ultra Durable Stain...,,4.4 out of 5,No Stock
3,"Fidget Spinner, Thanksgiving Christmas Led Lig...",,4.5 out of 5,No Stock
4,"Fidget Spinners, Fidget Spinner Toy Gifts for ...",,4.5 out of 5,No Stock
...,...,...,...,...
64,Rainbow Anti-Anxiety Fidget Spinner [Metal Fid...,,4.4 out of 5,No Stock
65,Cool Fidget Spinners Toy Metal for Kids Adults...,,4.4 out of 5,No Stock
66,"Jawhock 12 Pack Fidget Spinners, High Speed Ha...",,4.2 out of 5,No Stock
67,GetWill 24 Pack Party Favors Christmas Fidget ...,,4.7 out of 5,No Stock
